In [5]:
#!/usr/bin/env python3
# -*- : utf-8 -*-
"""
Created on Tue May 29 21:12:40 2018

@author: hielke & niels
"""

import numpy as np
from numpy import matlib
import random
from pprint import PrettyPrinter as pp
from matplotlib import pyplot as plt
pprint = pp().pprint

%matplotlib inline

# Summary
Hopfield networks are a type of autoencoder, which are able to reconstruct a complete dataset while only a limited amount of data is available. To achieve this, the network must first be trained on the undistorted data. 
In our code, we trained the networks as described below in "hebbian". 
A trained network will have one or multiple stable states. A random input state will converge to one of these states when it is repeatedly updated. Note that states must be binary to guarantee convergence. In this case neurons could be in the state -1 or 1.

For each to be learned configuration, a hebbian association matrix is constructed where matrix[i, j] is calculated by the product of the states of neuron i and j. This implies that a Hopfield network is symmetric: product i,j is equal to product j,i. Thus  weights[i, j] is equal to weights[j, i]. Furthermore, the elements for which i=j (i.e. the diagonal of the matrix) will be zero because a Hopfield network does not contain self-loops. Once such a matrix is constructed for each of the to be learned states, all matrices are added to make the weights matrix. This matrix is implemented in "hebbian" and "multiple_hebbian" below. 

The update procedure relies on energy minimalization. Energy is determined by the weights matrix. If changing the state of a neuron to 1 will decrease the total energy of the system, the state will be adapted. Otherwise, it will be -1. The change in energy is calculated as follows and is implemented in "update". The association of each neuron with the evaluated neuron j (i.e. weights[i, j] for i in range(number of neurons)) is multiplied by the state of neuron j. If the sum of these values is larger than the threshold associated with this neuron, the state will be set to 1.

In our code, the update rule is applied until convergence or until the system has been updated 1000 times. In the first case, the network successfully reconstructed one of the stable states; in the second case, it failed to do so.
Below, we describe our class "Hopfield" used to implement a Hopfield network.

# class Hopfield
## hebbian
This method constructs the hebbian association matrix (called "weights") given a vector that contains the state of each neuron:
First, the transpose of the vector is multiplied with the vector itself resulting in a matrix. Since a Hopfield network requires that a neuron is not directly connected to itself, the diagonal must be zero. 
Because the states are always either 1 or -1, the product of a neuron state with itself is always 1. Therefore, the diagonal can be reduced to zero by subtracting the identity matrix from the previously calculated matrix. 

## multiple_hebbian
Similar to hebbian described above, multiple_hebbian constructs the weights matrix. The difference is that this method takes a matrix of inputs (multiple, unique states) instead of a vector (a single state). 

## reset_states
Resets the states: randomizes the state of each neuron to either -1 or 1. So that different trials will be independent of each other.


## update
For every update, a random neuron is chosen with random.choice. If the sum of the weights of each connection with this neuron is larger than its respective threshold, the state will be set to 1. Otherwise it will be set to -1. 


## converge
This function updates the system until it converges or until a specified number of iterations has been concluded. After every update, the change of the updated neuron i is tracked. If its state is the same as before, changed_state[i] will be set to False. If all elements of changed_state are set to False, the system has converged. On the other hand, if the maximum number of iterations has been met, the system has not converged.

## list_attractors
This function keeps track of all attractor states (i.e. the states the system can converge to). The converge function is used 100 times independently. For each of these tries, the neurons are independently randomized.


In [6]:
class Hopfield:
    
    def __init__(self, weights):
        weights = np.asarray(weights)
        size = len(weights)
        assert size * size == np.size(weights), \
                    "Weights should be square matrix"

        self.weights = weights
        self.size = size
        
        self.reset_states()
        self.tresholds = np.zeros(size) 
        
        assert not (self.weights * np.eye(self.size)).all(), \
                "There are self-loops"
        assert np.allclose(self.weights, self.weights.T), \
                "Not symmetric"
        
    @classmethod
    def hebbian(cls, vector):
        size = np.size(vector)
        vector = np.asmatrix(vector)
        weights = vector.T * vector - np.eye(size)
        return cls(weights)
    
    @classmethod
    def multiple_hebbian(cls, matrix):
        size = np.size(matrix[0, :])
        amm_vectors = len(matrix)
        assert np.size(matrix) == size * amm_vectors, \
                "Not all vectors are of the same length"
        weights = np.matlib.zeros((size, size))
        for i in range(amm_vectors):
            vec = np.asmatrix(matrix[i, :])
            weights += vec.T * vec
        weights -= amm_vectors * np.matlib.eye(size)
        return cls(weights)
    
    @classmethod
    def with_tresh(cls, weights, tresh):
        hp = cls(weights)
        hp.tresholds = tresh
        return hp
        
    def reset_states(self):
        self.states = np.array([random.choice([-1, 1]) 
                for _ in range(self.size)], dtype=int)
    
    def update(self):
        i = random.choice(range(self.size))
        
        delta = 0
        for j in range(self.size):
            delta += self.weights[i, j] * self.states[j]
        
        new_state = 1 if delta >= self.tresholds[i] else -1
        
        changed = self.states[i] != new_state
        self.states[i] = new_state
        
        return i, changed
        
    def converge(self):
        changed_states = [True for _ in range(self.size)]
        counter = 0
        while any(changed_states):
            i, change = self.update()
            if change:
                changed_states = [True for _ in range(self.size)]
            else:
                changed_states[i] = False
            counter += 1
            if counter > 10000:
                return print("ERR: exceeded maximum iterations.")
        return self.states
    
    def list_attractors(self):
        attr_list = []
        
        for _ in range(100):
            self.reset_states()
            attr = self.converge()
            
            for a in attr_list:
                if np.array_equal(a, attr):
                    break
            else:
                attr_list.append(attr)
        
        return attr_list
    
    def plot_neuron_states(self, height, width, states=None):
        if states is None:
            states = self.states
        print(height, width, np.size(states))
        assert height * width == np.size(states), \
                "Height and width do not match the size of the states."
        
        image = []
        
        for h in range(height):
            image.append(states[h*width:(h+1)*width])
        plt.figure()
        plt.imshow(image, cmap='gray')

# Simple convergence
This function takes a trivial matrix of which the diagonals are 0 (thus implying that there are no self-connections) and feeds it to the Hopfield class. Consequently, the class method  "converge" is applied in the outer loop of a nested for loop. 


In [7]:
weights = np.matrix([[0, -1, 1],
                    [0, 0, -1],
                    [0, 0, 0]])
weights += weights.T
hp = Hopfield(weights)
print("attractors:")
pprint(hp.list_attractors())

attractors:
[array([-1,  1, -1]), array([ 1, -1,  1])]


# Simple Hebbian

In [8]:
vector = np.matrix([1, 1, 1, 1, 1, 1, 1, 1])
hp = Hopfield.hebbian(vector)
print("attractors:")
pprint(hp.list_attractors())

attractors:
[array([1, 1, 1, 1, 1, 1, 1, 1]), array([-1, -1, -1, -1, -1, -1, -1, -1])]


# Again a simple Hebbian

In [9]:
vector = np.matrix([1,1,-1,1])
hp = Hopfield.hebbian(vector)
print("attractors:")
pprint(hp.list_attractors())

attractors:
[array([ 1,  1, -1,  1]), array([-1, -1,  1, -1])]


# An example with multiple things to learn

In [10]:
matrix = np.array([[1, 1, 1, 1],
                   [1, -1, 1, -1],
                           [1, 1, -1, -1]])
hp = Hopfield.multiple_hebbian(matrix)
print("attractors:")
pprint(hp.list_attractors())

attractors:
[array([ 1, -1,  1, -1]),
 array([-1,  1, -1,  1]),
 array([ 1,  1, -1, -1]),
 array([-1, -1, -1, -1]),
 array([-1, -1,  1,  1]),
 array([1, 1, 1, 1])]


# Perceptron learning

In addition to Hebbian learning you can also transform your weights and desired output in a certain way to feed it to a perceptron learner. This is the tranformation:

In [5]:
def transformation(vector):
    size = len(vector)
    tres = np.ones((1,size))
    z = np.zeros((size, size + size*(size-1)//2))
    for i in range(size):
        ran = (size-1)*size//2 - (size-1-i)*(size-i)//2
        z[i,ran:ran+size-i-1] = vector[i+1:]
        z[i,-(size-i)] = tres[0,i]
        for j in range(i):
            z[i, ]
        j = i
        while j > 0:
            ran = (size-1)*size//2 -(size-j)*(size-j+1)//2
            z[i, i-1+ran] = vector[j-1]
            j -= 1
    return z

The next cell contains the function to learn the perceptron. It takes a matrix with the desired outputs as rows, and the outputs the weights and the tresholds.

In [18]:
def hopfield_perceptron(matrix):
    
    rate = .01 # The learning rate, increasing can cause faster convergence, 
               # but can also cause oscillations around a stable state.
    err = .4   # The error rate, increasing it, can stabilize the network more,
               # but increasing it too much can again cause oscillations around a stable state.
    
    vector = matrix[0, :]
    z = transformation(vector)
    sol = np.array([random.choice([-.01, .01]) for _ in range(len(z[0,:]))])
    
    amm_vectors = len(matrix[:, 0])
    
    for i in range(1000):
        still_wrong = False
        for j in range(amm_vectors):
            vector = matrix[j, :]
            z = transformation(vector)
            for i in range(len(vector)):
                if not sol.dot(z[i, :]) * vector[i] > err: # prediction incorrect
                    sol += rate * z[i, :] * vector[i]
                    still_wrong = True
        if not still_wrong:
            print("We are good")
            break
    else:
        print("We have not converged")
        
    triu = zip(*np.triu_indices(len(vector), 1))
    weights = np.matlib.zeros((len(vector), len(vector)))
    for ind, iu in enumerate(triu):
        weights[iu] = sol[ind]
    
    weights += weights.T

    tresh = -1 * sol[-len(vector):]
    
    return weights, tresh

### Simple example

In [14]:
matrix = np.array([[1, 1, -1, 1, -1, 1, -1, -1],
                       [1, -1, 1, -1, 1, -1, 1, 1]])
weights, tresh = hopfield_perceptron(matrix)
hp = Hopfield.with_tresh(weights, tresh)
attractors = hp.list_attractors()
pprint(attractors)

We are good
[array([ 1,  1, -1,  1, -1,  1, -1, -1]),
 array([ 1, -1,  1, -1,  1, -1,  1,  1])]


### An example with too much too learn

In this example the network must learn too much, and will therefore have incorrect attractors.

In [19]:
matrix = np.array([[1, 1, -1, 1, -1, 1, -1, -1],
                       [1, 1, -1, 1, -1, -1, -1, 1],
                       [1, 1, 1, 1, -1, -1, -1, -1],
                       [1, -1, 1, -1, 1, -1, 1, 1]])
weights, tresh = hopfield_perceptron(matrix)
hp = Hopfield.with_tresh(weights, tresh)
attractors = hp.list_attractors()
pprint(attractors)

We have not converged
[array([-1, -1, -1, -1,  1,  1,  1,  1]),
 array([ 1,  1,  1,  1, -1, -1, -1, -1]),
 array([ 1, -1,  1, -1,  1, -1,  1,  1]),
 array([ 1, -1, -1, -1, -1,  1,  1,  1])]


### Same problem with Hebbian learning

Interestingly, if we feed this same problem into the hebbian learning algorithm, it will find less attractors than the perceptron. So, the perceptron is closer to the solution, but still cannot find it.

In [20]:
matrix = np.array([[1, 1, -1, 1, -1, 1, -1, -1],
                   [1, 1, -1, 1, -1, -1, -1, 1],
                   [1, 1, 1, 1, -1, -1, -1, -1],
                   [-1, 1, -1, 1, -1, 1, -1, 1]])
hp = Hopfield.multiple_hebbian(matrix)
print("attractors:")
attractors = hp.list_attractors()
pprint(attractors)

attractors:
[array([-1, -1,  1, -1,  1,  1,  1,  1]),
 array([ 1,  1, -1,  1, -1,  1, -1,  1])]
